# Getting to Fast

Let's take a look at some variations on updating state in a LASER model.

The basic idea is to identify agents in our population who meet some criteria and then update their state. In general we can call this query-process.

Here are some baseline properties and values:

In [9]:
from time import perf_counter_ns
import numpy as np

count = 50_000_000  # About 10 seconds on 2-core Codespace
velocity = np.random.rand(count) * 20  # [0, 20)
state = np.zeros(count)
threshold = 10.0
timings = {}

## Naïve Python

First, let's just loop over our properties, check for the condition we want `velocity > threshold` and update the state when true (`state = 1`).

In [10]:
py_state = np.zeros(count)
t_start = perf_counter_ns()
for i in range(count):
    if velocity[i] > threshold:
        py_state[i] = 1
t_finish = perf_counter_ns()
elapsed = t_finish - t_start
print(f"Python for loop took {elapsed/1e6:12,.3f} ms")
timings["for loop"] = elapsed

Python for loop took   11,495.112 ms


## NumPy

Next, let's look at a common NumPy idiom.

NumPy should be _much_ faster than the naïve Python for loop. However, a drawback to this idiom with LASER models is that often we have millions of agents so the query expression, `velocity > threshold`, creates a temporary Boolean array with the same number of entries as agents. This can easily be many megabytes for country level models up to gigabytes for continental scale modeling. Multi-part expressions can be even larger with temporaries for the sub-parts, e.g. `(age > 5) and (vaccinated == 0)` which would allocate 3x the memory.

In [11]:
np_state = np.zeros(count)
t_start = perf_counter_ns()
np_state[velocity > threshold] = 1
t_finish = perf_counter_ns()
elapsed = t_finish - t_start
print(f"NumPy vectorized took {elapsed/1e6:12,.3f} ms")
print(f"{np.all(np_state == py_state)=}")
timings["NumPy"] = elapsed

NumPy vectorized took      316.748 ms
np.all(np_state == py_state)=np.True_


## Numba

Now, let's apply Numba to the challenge - compiled, parallelized code. Results will vary depending on the number of CPU cores available on the system.

An additional significant advantage of Numba compiled code is that the results of the query are never stored. This is appropriate for an ephemeral query which is calculated, used to identify target agents, and then discarded. This alone save megabytes of memory alocations and RAM bandwidth.

Numba compiled code is generally only useful for functions which take more than 1 second to complete since there is overhead to the compilation process. To simulate what _subsequent_ calls to our function would look like, we will first apply our select and update functions on a dummy array.

In [12]:
import numba as nb

@nb.njit(parallel=False)
def compiled_single(count, velocity, threshold, state):
    for i in nb.prange(count):
        if velocity[i] > threshold:
            state[i] = 1

compiled_single(16, velocity=np.random.rand(16)*20, threshold=10.0, state=np.zeros(count))

tmp_state = np.zeros(count)
t_start = perf_counter_ns()
compiled_single(count, velocity=velocity, threshold=threshold, state=tmp_state)
t_finish = perf_counter_ns()
elapsed = t_finish - t_start
print(f"Numba compiled_single took {elapsed/1e6:12,.3f} ms")
print(f"{np.all(tmp_state == py_state)=}")
timings["Numba (1 core)"] = elapsed

Numba compiled_single took       68.517 ms
np.all(tmp_state == py_state)=np.True_


In [13]:
@nb.njit(parallel=True)
def compiled_multi(count, velocity, threshold, state):
    for i in nb.prange(count):
        if velocity[i] > threshold:
            state[i] = 1

compiled_multi(16, velocity=np.random.rand(16)*20, threshold=10.0, state=np.zeros(count))

tmp_state = np.zeros(count)
t_start = perf_counter_ns()
compiled_multi(count, velocity=velocity, threshold=threshold, state=tmp_state)
t_finish = perf_counter_ns()
elapsed = t_finish - t_start
print(f"Numba compiled_multi took {elapsed/1e6:12,.3f} ms")
print(f"{np.all(tmp_state == py_state)=}")
timings["Numba (parallel)"] = elapsed

Numba compiled_multi took       29.577 ms
np.all(tmp_state == py_state)=np.True_


In [14]:
from laser_core.perf import apply

print(f"{nb.get_num_threads()=}")

def select(i, velocity, threshold):
    return velocity[i] > threshold

def update(i, state):
    state[i] = 1

apply(select, update, 16, velocity=np.random.rand(16) * 20, threshold=10.0, state=np.zeros(count))

nb_state = np.zeros(count)

t_start = perf_counter_ns()
apply(select, update, count, velocity=velocity, threshold=threshold, state=nb_state)
t_finish = perf_counter_ns()
elapsed = t_finish - t_start
print(f"Numba compiled took {elapsed/1e6:12,.3f} ms")
print(f"{np.all(nb_state == py_state)=}")
timings["Numba (apply)"] = elapsed

nb.get_num_threads()=8
Numba compiled took       27.863 ms
np.all(nb_state == py_state)=np.True_


## `numbafy()` - An Alternative to `apply()`

`apply()` could be a bit verbose for usage in multiple places, so there is an alternative equivalent, `numbafy()` which returns a function to be called later rather than executing the query+update immediately. In this example we will simulate using `numbafy()` to add functionality to a class.

In [15]:
from laser_core.perf import numbafy

class Component():
    def __init__(self):

        def select(i, velocity, threshold):
            return velocity[i] > threshold

        def update(i, state):
            state[i] = 1

        self.do_action = numbafy(select, update)

        return

component = Component()

# "Warm up" Numba
component.do_action(16, velocity=np.random.rand(16) * 20, threshold=10.0, state=np.zeros(16))

da_state = np.zeros(count)
t_start = perf_counter_ns()
component.do_action(count, velocity=velocity, threshold=threshold, state=da_state)
t_finish = perf_counter_ns()
elapsed = t_finish - t_start
print(f"Numbafy compiled took {elapsed/1e6:12,.3f} ms")
print(f"{np.all(da_state == py_state)=}")
timings["Numba (numbafy)"] = elapsed

Numbafy compiled took       20.441 ms
np.all(da_state == py_state)=np.True_


## Results

In [16]:
print(f"Iterating over {count:,} agents:")
for k, v in timings.items():
    print(f"{k:20}: {v/1e6:12,.3f} ms")

Iterating over 50,000,000 agents:
for loop            :   11,495.112 ms
NumPy               :      316.748 ms
Numba (1 core)      :       68.517 ms
Numba (parallel)    :       29.577 ms
Numba (apply)       :       27.863 ms
Numba (numbafy)     :       20.441 ms
